<a href="https://colab.research.google.com/github/ali-farshad/Homework1-ME5920/blob/main/3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install opencv-python-headless scikit-learn

import os, glob
import numpy as np
import cv2
from sklearn.cluster import KMeans

In [ ]:
def segment_green_mask(bgr):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

    lower = np.array([25, 40, 40])
    upper = np.array([95, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    return mask

In [ ]:
def get_centroids_from_mask(mask, min_area=80):
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

    pts = []
    for i in range(1, num):  # skip background
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            cx, cy = centroids[i]
            pts.append([cx, cy])

    return np.array(pts, dtype=np.float32)

In [ ]:
def kmeans_1d(values, k=6, random_state=0):
    values = values.reshape(-1, 1)
    km = KMeans(n_clusters=k, n_init=10, random_state=random_state)
    km.fit(values)
    centers = np.sort(km.cluster_centers_.ravel())
    return centers

def centers_to_edges(centers, vmin, vmax):
    centers = np.sort(centers)
    mids = (centers[:-1] + centers[1:]) / 2.0
    edges = np.concatenate([[vmin], mids, [vmax]])
    return edges